In [1]:
cnx = mysql.connector.connect(
    host="127.0.0.1",
    port=3306,
    user="root",
    password="vikram")
cur = cnx.cursor()
sql = "USE scrape;CREATE TABLE scrape_dataset (id VARCHAR(255), pokemon_url VARCHAR(255), title TEXT, sold_date  VARCHAR(255),sold_price VARCHAR(255), auction_type VARCHAR(255),payment_status VARCHAR(255),buyers_premium VARCHAR(255), timestamp BIGINT UNSIGNED, image_downloaded TINYINT, img_url_1 TEXT, img_url_2 TEXT); ALTER TABLE scrape_dataset ADD auto_id  INT PRIMARY KEY AUTO_INCREMENT;";
cur.execute(sql)




NameError: name 'mysql' is not defined

In [6]:
cnx = mysql.connector.connect(
    host="127.0.0.1",
    port=3306,
    user="root",
    password="vikram")
cur = cnx.cursor()
sql = "USE scrape;CREATE TABLE count (count BIGINT UNSIGNED , year INT UNSIGNED ,updated_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP)";
cur.execute(sql)

In [5]:
import mysql.connector

cnx = mysql.connector.connect(
    host="127.0.0.1",
    port=3306,
    user="root",
    password="vikram")
cur = cnx.cursor()
sql = "USE scrape;ALTER TABLE scrape_dataset ADD year INT UNSIGNED";
cur.execute(sql)

In [ ]:
year=2011
import mysql.connector
import json
from datetime import datetime
link_address='https://sales-history.pwccmarketplace.com/items/'
conn = mysql.connector.connect(
    host="127.0.0.1",
    port=3306,
    user="root",
    password="vikram",
    database="scrape"
)
cur = conn.cursor()



while year<2013:
    print('year===>',year)
    # changable
    f = open('apidata/'+str(year)+'_desc.json')
    data = json.load(f)
    # API data
    print(len(data))
    for pokemon in data:
        try:
            pokemon_url = link_address+pokemon['id']
            title = pokemon['title']
            sold_date = pokemon['soldDate']
            sold_price= pokemon['purchasePrice']
            auction_type=pokemon['auctionType']
            payment_status=pokemon['paymentStatus']
            buyers_premium='w/ Buyers Premium'
            pokemon_id=pokemon['id']
            sold_year = (sold_date[0:4])
            raw_date_year = (sold_date[:sold_date.index("T")])
            sold_date = raw_date_year.split("-",1)[1]+ '-' +sold_year
            epoch = int(datetime.strptime(sold_date, "%m-%d-%Y").timestamp())
            sql = "INSERT INTO scrape_dataset (id,pokemon_url,title,sold_date,sold_price,auction_type,payment_status,buyers_premium, timestamp, image_downloaded, img_url_1, img_url_2, year) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            val = (pokemon_id,pokemon_url,title,sold_date,sold_price,auction_type,payment_status,buyers_premium, epoch, 0,'','',year)
            cur.execute(sql, val)
            conn.commit()
    #         print(pokemon_id,pokemon_url,title,sold_date,sold_price,auction_type,payment_status,buyers_premium, epoch, image_downloaded,'','')
        except Exception as e:
            print('error occued')
            print(pokemon)
            print(e)
    year+=1
conn.close()    
f.close()


    # NON API DATA

    # for pokemon in data:
    #     try:
    #         pokemon_url = pokemon['pokemon_url']
    #         title = pokemon['title']
    #         sold_date = pokemon['sold_date'].replace('SOLD ON ', '')
    #         sold_price=pokemon['sold_price']
    #         auction_type=pokemon['auction_type']
    #         payment_status=pokemon['payment_status']
    #         buyers_premium=pokemon['buyers_premium']
    #         pokemon_id=pokemon_url.rsplit('/', 1)[-1]
    #         epoch = int(datetime.strptime(sold_date, "%m/%d/%Y").timestamp())
    #         sql = "INSERT INTO scrape_data (id,pokemon_url,title,sold_date,sold_price,auction_type,payment_status,buyers_premium, timestamp, image_downloaded, img_url_1, img_url_2) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    #         val = (pokemon_id,pokemon_url,title,sold_date,sold_price,auction_type,payment_status,buyers_premium, epoch, image_downloaded,'','')
    #         cur.execute(sql, val)
    #         conn.commit()
    #     except Exception as e:
    #         pokemon_url= link_address+ pokemon['id']
    #         title = pokemon['title']
    #         sold_date = pokemon['sold_date']
    #         sold_price=pokemon['sold_price']
    #         auction_type=pokemon['auction_type']
    #         payment_status=pokemon['payment_status']
    #         buyers_premium=pokemon['buyers_premium']
    #         pokemon_url=''
    #         cur.execute("SELECT * from data")
    #         print(e)

    # conn.close()    
    # f.close()


In [10]:
# Step 3:Images download
import mysql.connector
import json
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time
import requests

def configure_chrome_driver():
    driver = webdriver.Chrome(executable_path="/usr/bin/chromedriver")
    return driver

driver = configure_chrome_driver()


login_url = 'https://sales-history.pwccmarketplace.com/login?title=pokemon'
driver.get(login_url)
driver.find_element_by_id('email').send_keys('vikram.reddy.nov@gmail.com')
driver.find_element_by_id('password').send_keys('Vikked007#pwcc')
driver.find_element_by_class_name('custom-1ujrmx').click()
print('reached 1')
time.sleep(5)

conn = mysql.connector.connect(
    host="127.0.0.1",
    port=3306,
    user="root",
    password="vikram",
    database="scrape"
)


# only changable things year,iterations, limit
year=2020
rows=1
limit=20
iterations=0

while(iterations<125 and rows>0):
    sql_select_Query = "select id,auto_id from scrape_dataset where image_downloaded=0 and img_url_1='' and year="+ str(year) +" limit "+ str(limit)
    cursor = conn.cursor()
    cursor.execute(sql_select_Query)
    records = cursor.fetchall()
    rows=cursor.rowcount
    print('rows==>', rows)
    success_rows=0
    for row in records:
        id=row[0]
        auto_id=row[1]
        pokemon_link='https://sales-history.pwccmarketplace.com/items/'+id
        driver.get(pokemon_link)
#         time.sleep(3)
        try:
            front_image = driver.find_elements_by_xpath('//div[@class="custom-57bysn"]/img')[0].get_attribute("src")
            back_image = driver.find_elements_by_xpath('//div[@class="custom-57bysn"]/img')[1].get_attribute("src")
            front_image_data = requests.get(front_image).content
            back_image_data = requests.get(back_image).content
            with open('images/'+ str(year) +'//'+id+'_front', 'wb') as handler:
                handler.write(front_image_data)
            with open('images/'+ str(year) +'//'+id+'_back', 'wb') as handler:
                handler.write(back_image_data)

            sql = "UPDATE scrape_dataset SET image_downloaded=1, img_url_1='" + str(year)+ '/' + id+"_front'" + ",img_url_2='" + str(year) + '/' +id+"_back'" + " where auto_id="+str(auto_id)
            cur = conn.cursor()
            cur.execute(sql)
            conn.commit()
            success_rows+=1
        except Exception as e:
            print(e)
            print('failed',id)
            sql = "UPDATE scrape_dataset SET img_url_1='3' where auto_id="+ str(auto_id)
            cur = conn.cursor()
            cur.execute(sql)
            conn.commit()
            
    sql = "INSERT INTO count (count, year) VALUES(%s,%s)"
    val=(success_rows, year)
    cur = conn.cursor()
    cur.execute(sql,val)
    conn.commit()
    iterations+=1
    


conn.close()    

reached 1
rows==> 2
list index out of range
failed WEEKLY3571169
rows==> 0
